In [1]:
import transformers
from transformers import pipeline

In [2]:
pip install triton

Kurulumdan sonra, çalışma zamanını (runtime) yeniden başlatmanız gerekebilir (yukarıdaki menüden `Runtime > Restart runtime` seçeneğini kullanarak) veya bazen kütüphanenin doğru şekilde algılanması için tüm notebook'u yeniden çalıştırmanız gerekebilir. Yeniden başlattıktan sonra, `transformers` ve `pipeline` import eden hücreyi tekrar çalıştırın ve uyarının devam edip etmediğini kontrol edin.

In [3]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
sentence_1 = "i had a great time at the movie it was really funny"
sentence_2 = "i had a great time at the movie but the parking was terrible"
sentence_3 = "i had a great time at the movie but the parking wasn't great"
sentence_4 = "i went to see a movie"

In [5]:
print(sentence_1)
sentiment_pipeline(sentence_1)

i had a great time at the movie it was really funny


[{'label': 'POSITIVE', 'score': 0.9998176693916321}]

In [7]:
test = sentiment_pipeline(sentence_1)
[sub['label'] for sub in test]

['POSITIVE']

In [8]:
print(sentence_2)
sentiment_pipeline(sentence_2)

i had a great time at the movie but the parking was terrible


[{'label': 'NEGATIVE', 'score': 0.9977464079856873}]

In [9]:
print(sentence_3)
sentiment_pipeline(sentence_3)

i had a great time at the movie but the parking wasn't great


[{'label': 'NEGATIVE', 'score': 0.9984902143478394}]

In [10]:
print(sentence_4)
sentiment_pipeline(sentence_4)

i went to see a movie


[{'label': 'POSITIVE', 'score': 0.9802700281143188}]

The default model isn't giving us great results for our neutral sentence. However there are loads of models we can choose to use that have been trained on different data with different parameters: https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads&search=sentiment

Let's try a specific model trained on tweets in the English language only and see if that improves our results.

In [11]:
specific_model = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use cpu


In [12]:
print(sentence_1)
specific_model(sentence_1)

i had a great time at the movie it was really funny


[{'label': 'POS', 'score': 0.9923344254493713}]

In [13]:
print(sentence_2)
specific_model(sentence_2)

i had a great time at the movie but the parking was terrible


[{'label': 'NEG', 'score': 0.5355538129806519}]

In [14]:
print(sentence_3)
specific_model(sentence_3)

i had a great time at the movie but the parking wasn't great


[{'label': 'POS', 'score': 0.623440682888031}]

In [15]:
print(sentence_4)
specific_model(sentence_4)

i went to see a movie


[{'label': 'NEU', 'score': 0.9007399082183838}]

1. "Binary" (İkili) vs "Multi-class" (Çoklu) Sınıflandırma
Default Model (DistilBERT): Sadece "Pozitif" veya "Negatif" seçeneklerine sahip. Bu yüzden nötr bir cümle gördüğünde (Sentence 4), onu ya çok iyi ya da çok kötü diye etiketlemek zorunda kalıyor. Sonuçta 0.98 gibi yüksek bir skorla yanılıyor.

Özel Model (BERTweet): İçinde "Nötr" (NEU) seçeneği de var. Bu sayede duygusuz metinleri başarıyla ayırt edebiliyor.

Ders: Eğer gerçek dünya verileriyle (nötr cümlelerin bol olduğu) çalışıyorsan, mutlaka nötr sınıfı olan bir model seçmelisin.

2. Domain (Alan) Uyumu
BERTweet, Twitter verileriyle eğitilmiştir. Bu yüzden kısa, gündelik ve "ama/fakat" içeren karmaşık sosyal medya dilini anlamada, standart bir kitap/makale setinde eğitilmiş modelden daha başarılıdır.

Ders: Modeli nerede kullanacaksan (finans haberleri mi, tweetler mi, akademik makaleler mi?), o alanda eğitilmiş bir "pre-trained" model seçmek başarını %20-%30 artırır.

3. "Confidence Score" (Güven Skoru) Aldatıcı Olabilir
İlk modelin nötr cümleye %98 pozitif demesi, modelin çok emin ama tamamen haksız olduğunu gösteriyor.

Ders: Yüksek olasılık skorları her zaman "doğru sonuç" demek değildir. Modelin eğitildiği sınıfların dışına çıkıldığında bu skorlar anlamsızlaşır.

4. Karmaşık Cümlelerde (Sentence 3) Kararsızlık
Sentence 3: "I had a great time at the movie but the parking wasn't great"

Burada ilginç bir durum var: İlk model buna %99 negatif derken, ikinci model (BERTweet) %62 pozitif diyor.

Neden? Çünkü cümlenin yarısı çok iyi, yarısı kötü. Bir model "but"tan sonrasına odaklanırken, diğeri cümledeki "great" kelimelerinin ağırlığına takılabiliyor.

Ders: Hiçbir model %100 kusursuz değildir. Çok kritik projelerde birden fazla modelin ortalamasını almak (Ensemble) daha güvenli sonuç verir.